In [7]:
import numpy as np
import tensorflow as tf

from keras.layers import Input,Dense,Activation
from keras.layers import Dropout,BatchNormalization
from keras.layers import Concatenate
from keras.models import Model

import keras.backend as K

# Defining the Model for Simple-RNN

### Transformation for one time step

In [8]:
def simpleRNN_oneStep(input_shape,X_t,ht_prev,step_no):
    '''Arguments:
        input_shape: for this time step ie. (None,ToBeDecided)
        ht_prev    : the hidden layer output from the previous time-step
        step_no    : (for naming purpose) The current time-step number
    '''
    #Tunable(AK)
    #the internal normal feed forward dimension to create representation for next time step
    hidden_layer_dims=[1,]#To be decided
    output_layer_dim=1#To be decided
    rnn_initializer='glorot_uniform'
    
    #Making a place holder for current layer input
    X=Concatenate()([X_t,ht_prev]) #Concatenating it to the previous time-step memory
    
    #Implementing the current layers
    for i,dim in enumerate(hidden_layer_dims):
        X=Dense(dim,activation='relu',kernel_initializer=rnn_initializer,name='time'+str(step_no)+'hidden_'+str(i))(X)
        X=BatchNormalization(axis=-1)(X)
    
    #Final Layer that will go to next time step.(Currently just passing one layer activation to next step)
    #activation used is 'relu' cuz the packet loss or other metric later will be positive
    X=Dense(output_layer_dim,activation='relu',name='time'+str(step_no)+'passing')(X)
    
    return X

### Recursing for the final Model

In [9]:
def get_simpleRNN_model(input_shape,time_steps):
    '''Arguments:
        input_shape: same as above.
        h_initial  : initial pre-contition i.e the starting state of the system
        time_steps : total time-steps in the recurrent sequence before thhe final output.
        
        This RNN is of type many-to-one architecture where the
        the input is taken at each time step-discretization of PerfSonar
        but output is generated after certain discrete RNN step(in paper they took
        referrent period of 24 hour time-step.)
    '''
    X_inputs=[]
    h_initial=Input(shape=(1,),dtype=tf.float32,name='initial_input')
    X_inputs.append(h_initial)
    for i in range(time_steps):
        X_t=Input(input_shape,dtype='float32',name='Input_time'+str(i+1))
        X_inputs.append(X_t)
    
    #Traversing through the timestep to create the recurrent effect
    for t,X_t in enumerate(X_inputs):
        if(t==1):
            X=simpleRNN_oneStep(input_shape,X_t,X_inputs[0],t)
        elif(t>1):
            X=simpleRNN_oneStep(input_shape,X_t,X,t)
            
    #This final time-step's NN's output is our prediction of the network parameters based on
    #previous "Referrent Time's  input"
    
    model=Model(inputs=X_inputs,outputs=X,name='RNN-V0')
    
    return model

# Testing the RNN

### Creating sample dataset

In [14]:
X=[]
Y=np.empty((1000,1))
time_step=100
for i in range(time_step+1):
    if(i==0):
        h_initial=np.zeros((1000,1))
        X.append(h_initial)
    else:
        xt_temp=np.zeros((1000,1))
        for j in range(1000):
            xt_temp[j,0]=100*j+(i-1)
        X.append(xt_temp)
for i in range(1000):
    Y[i,0]=100*(i+1)
    
    
#print(X[2][:,0])
#X_train=

### Compiling the Model

In [12]:
input_shape=(1,)
model=get_simpleRNN_model(input_shape,time_steps=100)
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

### Running the Training on Sample Test Set

In [13]:
model.fit(X,Y,epochs=10000,batch_size=1000) #dont keep the name as X and Y (kahe confuse kar rahe ho)

Epoch 1/6
1000/1000 [==============================] - 10s 10ms/step - loss: 3338335360.0000 - acc: 0.0000e+00
Epoch 2/6
1000/1000 [==============================] - 0s 115us/step - loss: 3338335104.0000 - acc: 0.0000e+00
Epoch 3/6
1000/1000 [==============================] - 0s 142us/step - loss: 3338335360.0000 - acc: 0.0000e+00
Epoch 4/6
1000/1000 [==============================] - 0s 113us/step - loss: 3338335360.0000 - acc: 0.0000e+00
Epoch 5/6
1000/1000 [==============================] - 0s 130us/step - loss: 3338334848.0000 - acc: 0.0000e+00
Epoch 6/6
1000/1000 [==============================] - 0s 112us/step - loss: 3338334848.0000 - acc: 0.0000e+00
